# Model Analysis

In [2]:
import pickle
from glob import glob
from pathlib import Path
import scipy.sparse as sp
from tqdm.notebook import tqdm
from gensim.models.word2vec import Word2Vec
from packages.TPPMI.ppmi_model import PPMIModel
from packages.TPPMI.tppmi_model import TPPMIModel

from embedding_visualization import  \
    plot_word_vectors_cade,  plot_word_vectors_tppmi, print_most_similar_cade

## Setup

### Paths

In [3]:
model_path = Path("model/")
number_of_context_words = 500
ppmi_path = Path(f"data/ppmi-matrices/quarter/{number_of_context_words}")

### Load Models

In [4]:
cade_model_filenames = glob(str(model_path / "quarter" / "*.model"))

In [5]:
# load models
cade_models_quarterly = {f"model_{model_file.split('_')[1][0:3].lower()}":Word2Vec.load(model_file) for model_file in tqdm(cade_model_filenames)}

  0%|          | 0/4 [00:00<?, ?it/s]

Put them in the right chronological order

In [6]:
order = ['jun', 'sep', 'dec', 'mar', 'apr']
month_codes = {"jun": 6, "sep": 9, "dec": 12, "mar": 3}

# Sort the keys based on the desired month order
sorted_keys = sorted(cade_models_quarterly.keys(), key=lambda x: order.index(x.split('_')[1]))

# Create a sorted dictionaries
cade_models_quarterly = {key: cade_models_quarterly[key] for key in sorted_keys}

In [7]:
cade_models_quarterly.keys()

dict_keys(['model_jun', 'model_sep', 'model_dec', 'model_mar'])

### TPPMI

In [8]:
ppmi_data_files = sorted(glob(str(ppmi_path  / "*.npz")))
words_files = sorted(glob(str(ppmi_path  / "*.pkl")))

Split context-words from timestamped-vocabularies

In [9]:
context_words_file = [path for path in words_files if "context-words" in path]
ppmi_vocab_files = [path for path in words_files if "context-words" not in path]

In [10]:
# Get ppmi-matrices and vocab
ppmi_matrices = {}

for filenames in zip(ppmi_vocab_files, ppmi_data_files):
    ppmi_matrix = sp.load_npz(filenames[1])
    with open(filenames[0], "rb") as f:
        vocab = pickle.load(f)
    key = filenames[0].split("ppmi-")[2][0:2]
    ppmi_matrices[key] = {"ppmi_matrix" : ppmi_matrix, "vocab": vocab}

# Get common context-words
with open(context_words_file[0], "rb") as f:
    context_words = pickle.load(f)

In [11]:
ppmi_matrices.keys()

dict_keys(['03', '06', '09', '12'])

Create ppmi_model objects

In [12]:
ppmi_models = {key: PPMIModel.construct_from_data(ppmi_data["ppmi_matrix"], ppmi_data["vocab"], context_words) for key, ppmi_data in ppmi_matrices.items()}

In [13]:
tppmi_model = TPPMIModel(ppmi_models)

## Covid

In [14]:
print_most_similar_cade(cade_models_quarterly, "lockdowns", top_n=10)

Word: lockdowns
Month: Jun
[('disruptions', 0.815423846244812),
 ('eased', 0.8148480653762817),
 ('restrictions', 0.8083637356758118),
 ('closures', 0.8061288595199585),
 ('lockdown', 0.7959799766540527),
 ('closure', 0.7941170930862427),
 ('curbs', 0.7778494358062744),
 ('disruption', 0.763401210308075),
 ('quarantines', 0.7625197172164917),
 ('protocols', 0.761249303817749)]
--------------------------------
Month: Sep
[('shutdowns', 0.8916682004928589),
 ('closings', 0.8569907546043396),
 ('draconian', 0.8482598066329956),
 ('mitigations', 0.8216508030891418),
 ('covidrelated', 0.8044072985649109),
 ('closures', 0.8002327084541321),
 ('restrictions', 0.7890362739562988),
 ('strict', 0.7837325930595398),
 ('lockdown', 0.7757399678230286),
 ('precautions', 0.7519178986549377)]
--------------------------------
Month: Dec
[('closures', 0.8628657460212708),
 ('shutdowns', 0.859953761100769),
 ('shutdown', 0.8257884383201599),
 ('closings', 0.8155633211135864),
 ('protocols', 0.80748170614

## CRT

In [15]:
print_most_similar_cade(cade_models_quarterly, "crt", top_n=3)

Word: crt
Month: Jun
[('marxist', 0.6683539748191833),
 ('marxism', 0.6651936173439026),
 ('socialism', 0.6600244641304016)]
--------------------------------
Month: Sep
[('wokeism', 0.7039192914962769),
 ('garbage', 0.6873442530632019),
 ('reality', 0.6830105781555176)]
--------------------------------
Month: Dec
[('crts', 0.6683191061019897),
 ('word', 0.6668873429298401),
 ('screen', 0.6599460244178772)]
--------------------------------
Month: Mar
[('modern', 0.7213348746299744),
 ('word', 0.7110840678215027),
 ('lcd', 0.6734970808029175)]
--------------------------------


## Mass Shootings

### Overview

- **May:** Uvalde Robb Elementary School shooting (May 24, 2022)
- **November:** Colorado Springs nightclub shooting (November 19, 2022)
- **March:** Nashville school shooting (March 27, 2023)

In [16]:
target_word_shootings = "shooting"
test_words_shootings = ["uvalde", "colorado", "nashville"]

In [17]:
title_shootings = f"Evolution of the word {target_word_shootings} over time"

### Visualizations

#### Cade

In [18]:
plot_word_vectors_cade(cade_models_quarterly, test_words_shootings + [target_word_shootings], range=[-2, 2], use_tsne=True, perplexity=3)

#### TPPMI

In [19]:
tppmi_shootings = tppmi_model.get_tppmi(test_words_shootings + [target_word_shootings], smooth=True)

All words are contained in the vocabulary


In [21]:
plot_word_vectors_tppmi(tppmi_model, test_words_shootings + [target_word_shootings],
                        use_tsne=False, range=[-15, 15])

All words are contained in the vocabulary


In [22]:
tppmi_shootings = tppmi_model.get_tppmi(test_words_shootings + [target_word_shootings])

All words are contained in the vocabulary


## Elon Musk Twitter takeover

### Overview

Business magnate Elon Musk initiated an acquisition of American social media company Twitter, Inc. on April 14, 2022, and concluded it on October 27, 2022. Musk had begun buying shares of the company in January 2022, becoming its largest shareholder by April with a 9.1 percent ownership stake

source: [Wikipedia article](https://en.wikipedia.org/wiki/Acquisition_of_Twitter_by_Elon_Musk#:~:text=Business%20magnate%20Elon%20Musk%20initiated,a%209.1%20percent%20ownership%20stake.)

In [23]:
target_word_twitter = "twitter"
test_words_twitter = ["facebook",  "elon", "instagram"]

In [24]:
title_twitter = f"Evolution of the word {target_word_twitter} over time"

### Visualizations

In [25]:
plot_word_vectors_cade(cade_models_quarterly, test_words_twitter + [target_word_twitter], range=[-8, 8])

In [26]:
tppmi_twitter = tppmi_model.get_tppmi(test_words_twitter + [target_word_twitter])

All words are contained in the vocabulary


In [28]:
plot_word_vectors_tppmi(tppmi_model, test_words_shootings + [target_word_shootings], range=[-10, 20])

All words are contained in the vocabulary


## President

### Overview

June:
- Boris Johnson: vote of no confidence June 6, 2022
- Lyndon B Johnson: Talks leading up to the civil rights act July 2, (1964)
- Janet Godwin: The Iowa City-based education testing company ACT will be laying off 106 employees by July 1.

September:
- Scott Hagel: Vice president of marketing

January:
- Richard Cordoray: Considered as vice chairman of the federal reserve for banking supervision

March/April:
- Donald Trump: Trump charged with 34 felony counts in hush money scheme

In [27]:
target_word_president = "president"
test_words_president = ["johnson", "trump", "biden", "former", "obama", "minister"]

### Visualizations

In [ ]:
plot_word_vectors_cade(cade_models_quarterly, test_words_president + [target_word_president], range=[-10, 10])

In [ ]:
tppmi_president = tppmi_model.get_tppmi(test_words_president + [target_word_president])

In [ ]:
plot_word_vectors_tppmi(tppmi_model.get_2d_representation(test_words_president + [target_word_president]), range=[-10, 15])